In [1]:
from tqdm import tqdm
import math
from pathlib import Path
from fastai import *
from fastai.vision.all import *
from fastai.data.all import *
from pathlib import Path
from PIL import Image
import math
from IPython.display import display
import torch

In [30]:
before_tensor_path = "../data/before_after_images/features/before/list_of_tensors.pt"
after_tensor_path = "../data/before_after_images/features/after/list_of_tensors.pt"

In [39]:
img_dir = Path("../data/before_after_images/processed")

before_dir = img_dir / 'before'
after_dir = img_dir / 'after'

In [2]:
learn_inf = load_learner("../saved_model/before_after.pkl")

In [12]:
def predict(fn, scale=1):
    sample_dir = Path("../data/sample_images")
    path = sample_dir / fn
    
    return learn_inf.predict(path)

In [3]:
input_tensor = []

In [4]:
def feature_extract_hook(m, i, o):
    input_tensor.append(i)

In [7]:
# pred = predict("sample.jpeg")

In [29]:
def get_tensors(file_dir):
    # model[-1] -> last block of resnet 18
    # model[-1][-1] -> Last linear 512 -> 1 layer
    hook = learn_inf.model[-1][-1].register_forward_hook(feature_extract_hook)
    test_files = [fn for fn in sorted((file_dir).glob('**/*')) if fn.is_file() and fn.name != ".gitkeep"]
    test_dl = learn_inf.dls.test_dl(test_files)
    preds = learn_inf.get_preds(dl=test_dl)
    
    hook.remove()
    
    return torch.stack([t for batch in input_tensor for sample in batch for t in sample])
    

In [15]:
flat_list_tensors = get_tensors(after_dir)
flat_list_of_tensors.shape

torch.Size([1292, 512])

In [16]:
torch.save(flat_list_of_tensors, tensor_path)

In [18]:
last_layer = learn_inf.model[-1][-1]

In [23]:
one_tensor = flat_list_of_tensors[0].unsqueeze(0)

In [25]:
one_tensor.shape

torch.Size([1, 512])

In [24]:
last_layer(one_tensor)

TensorBase([[ 1.6057, -0.6762]], grad_fn=<AliasBackward0>)

## New session, loading saved tensor

In [57]:
sm = torch.nn.Softmax()

def predict_with_feature(input_tensor):
    last_layer = learn_inf.model[-1][-1]
    res = sm(last_layer(input_tensor))
    label = "after" if res[0][0] > 0.5 else "before"
    
    return label, res
    

In [66]:
test_files = [fn for fn in sorted((before_dir).glob('**/*')) if fn.is_file() and fn.name != ".gitkeep" and "checkpoint" not in fn.name]

In [67]:
len(test_files)

1292

In [74]:
list_of_tensors = torch.load(before_tensor_path)

In [70]:
list_of_tensors.shape

torch.Size([1292, 512])

In [83]:
n = -10

In [84]:
learn_inf.predict(test_files[n])

('after', TensorBase(0), TensorBase([0.6974, 0.3026]))

In [85]:
predict_with_feature(list_of_tensors[n].unsqueeze(0))

('after', TensorBase([[0.6974, 0.3026]], grad_fn=<AliasBackward0>))